# 🎬 OneWord AI - Subtitle Generator

Generate viral-style one-word subtitles using AI!

**Perfect for**: Reels, Shorts, TikToks 🚀

---

## 📦 Step 1: Install & Check Version

We need **v0.1.3** or higher to work correctly.

In [ ]:
!pip install -q oneword-ai ffmpeg-python

import pkg_resources
ver = pkg_resources.get_distribution("oneword-ai").version
print(f"✅ Installed Version: {ver}")

if ver < "0.1.3":
    print("⚠️ OLD VERSION DETECTED! Please restart Runtime > Restart session, then run this cell again.")
else:
    print("✨ You have the fixed version!")

## 💾 Step 2: Connect Google Drive (Optional)

Save files directly to your Drive to avoid download issues.

In [ ]:
try:
    from google.colab import drive
    import os
    print("💾 Connecting to Drive...")
    drive.mount('/content/drive')
    output_dir = '/content/drive/MyDrive/OneWordAI_Outputs'
    os.makedirs(output_dir, exist_ok=True)
    print(f"✅ Connected! Saving to: {output_dir}")
except ImportError:
    print("⚠️ Drive not available (running locally?)")
    output_dir = "."

## 🌐 Option A: Easy Web UI (Gradio)

Best way to run the app. Click the public link below:

In [ ]:
import gradio as gr
from onewordai.core.engine import SubtitleGenerator
import tempfile
import os
from pathlib import Path

SAVE_DIR = output_dir if 'output_dir' in locals() else "."

def process_video(video_file, model, language, mode, progress=gr.Progress()):
    if video_file is None: return None, "❌ Upload a file first"
    try:
        progress(0, desc="Initializing...")
        generator = SubtitleGenerator(model_name=model)
        
        original_name = Path(video_file).stem
        srt_filename = f"{original_name}_{mode.lower().replace(' ', '')}.srt"
        output_path = os.path.join(SAVE_DIR, srt_filename)
        
        def show_progress(percent):
            progress(percent / 100, desc=f"Processing... {percent}%")
        
        result = generator.process(
            input_path=video_file,
            output_path=output_path,
            language=None if language == "Auto Detect" else language.lower()[:2],
            mode=mode.lower().replace(" ", ""),
            progress_callback=show_progress
        )
        
        progress(1.0, desc="Complete!")
        msg = f"✅ Saved to Drive: {srt_filename}" if '/content/drive' in str(output_path) else "✅ Download below"
        return result, msg
    except Exception as e:
        return None, f"❌ Error: {str(e)}"

with gr.Blocks(title="OneWord AI") as app:
    gr.Markdown("# 🎬 OneWord AI Subtitle Generator")
    with gr.Row():
        with gr.Column():
            video_input = gr.File(label="Upload Video", file_types=["video", "audio"])
            model_input = gr.Radio(["medium", "large", "Oriserve/Whisper-Hindi2Hinglish-Prime"], value="medium", label="Model")
            language_input = gr.Dropdown(["Auto Detect", "English", "Hindi", "Urdu", "Spanish"], value="Auto Detect", label="Language")
            mode_input = gr.Radio(["One Word", "Two Word", "Phrase"], value="One Word", label="Mode")
            btn = gr.Button("🚀 Generate", variant="primary")
        with gr.Column():
            out_file = gr.File(label="Download SRT")
            status = gr.Textbox(label="Status", interactive=False)
    btn.click(process_video, [video_input, model_input, language_input, mode_input], [out_file, status])

app.launch(share=True)

## 💻 Option B: Python Code

In [ ]:
from google.colab import files
uploaded = files.upload()
input_file = list(uploaded.keys())[0]

from onewordai.core.engine import SubtitleGenerator
from pathlib import Path
import os

SAVE_DIR = output_dir if 'output_dir' in locals() else "."
generator = SubtitleGenerator(model_name='medium')
output_filename = f"{Path(input_file).stem}_oneword.srt"
final_path = os.path.join(SAVE_DIR, output_filename)

print("Generating...")
generator.process(input_file, final_path)
print(f"✅ Done: {final_path}")

if SAVE_DIR == ".": files.download(final_path)

## 🤓 Option C: Run Your Own Server (Advanced)

Runs the `oneword-web` command and exposes it via Localtunnel.

In [ ]:
!npm install -g localtunnel

import subprocess
import threading
import time

# Start the FastAPI server in the background
def start_server():
    !oneword-web

threading.Thread(target=start_server, daemon=True).start()
time.sleep(5)  # Wait for boot

print("🌐 Starting Tunnel... Click the URL below:")
!lt --port 8000